# 飞桨常规赛：黄斑中央凹定位（GAMMA挑战赛任务二） - 12月第3名方案

## 本项目为2021.12参赛作品，最终得分8.53045
## 任务简要介绍
对2D眼底图像中黄斑中央凹进行定位。本任务的目的是预测黄斑中央凹在图像中的坐标值。若图像中黄斑中央凹不可见，坐标值设为(0, 0)，否则需预测出中央凹在图像中的坐标值。
## 赛题链接
https://aistudio.baidu.com/aistudio/competition/detail/120/0/introduction
[常规赛：黄斑中央凹定位（GAMMA挑战赛任务二）](https://aistudio.baidu.com/aistudio/competition/detail/120/0/introduction)


# 一、环境安装及准备数据
## 1.1 获取数据

In [1]:
! wget https://dataset-bj.cdn.bcebos.com/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/task2_Fovea_localization.zip

--2021-12-30 15:03:24--  https://dataset-bj.cdn.bcebos.com/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/task2_Fovea_localization.zip
Resolving dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)... 101.72.203.35, 123.125.132.35
Connecting to dataset-bj.cdn.bcebos.com (dataset-bj.cdn.bcebos.com)|101.72.203.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115329864 (110M) [application/zip]
Saving to: ‘task2_Fovea_localization.zip’

task2_Fovea_localiz 100%[===================>] 109.99M  41.7MB/s    in 2.6s    

2021-12-30 15:03:27 (41.7 MB/s) - ‘task2_Fovea_localization.zip’ saved [115329864/115329864]



In [2]:
! unzip -oq /home/aistudio/task2_Fovea_localization.zip

## 1.2 导入相关包

In [3]:
### 导入包

import sys 
sys.path.append('/home/aistudio/external-libraries')
import os
import cv2
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances 
import matplotlib.pylab as plt

import paddle
import paddle.nn as nn
import paddle.vision.models
from paddle.vision.models import resnet50
from paddle.io import Dataset


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


# 二、准备环节
## 2.1 文件重命名

In [4]:
# 统一一些命名
try:
    os.rename('training/fundus color images','training/fundus_image')
    os.rename('training/fovea_localization_training_GT.xlsx','training/Fovea_Location_train.xlsx')
    os.rename('testing/fundus color images','testing/fundus_image')
except:
    print("nothing to run")

## 2.2 配置参数
* 根据GPU显存大小将批次大小(Batch size)设定为32
* 将迭代轮数设定为3000，初始学习率设定为0.0001
* 本案例使用adam优化器
* 经测试以上参数设定可以达到较好的结果

In [5]:
### 设置参数

path = 'training/'  # 训练数据路径
images_file = path + 'fundus_image/'
gt_file = path + 'Fovea_Location_train.xlsx'
test_file = 'testing/fundus_image/'  # 测试数据路径
image_size = 256 # 输入图像统一尺寸 (image_size, image_size, 3)
val_ratio = 0.2 # 训练/验证数据划分比例
BATCH_SIZE = 32  # 批大小
iters = 3000 # 迭代次数
optimizer_type = 'adam' # 优化器，选手可选用其他优化器，如SGD, RMSprop,...
num_workers = 4 # 加载数据机器数
init_lr = 0.0001 # 初始学习率

# 三、数据处理
## 3.1 划分数据集
* 根据以上参数设定，以0.2比率划分训练集和验证集，数据集中有80%为训练集，有20%为验证集

In [6]:
### 从训练数据中划分训练集和验证集 

filelists = os.listdir(images_file)
train_filelists, val_filelists = train_test_split(filelists, test_size = val_ratio,random_state = 42)
print("Total Nums: {}, train: {}, val: {}".format(len(filelists), len(train_filelists), len(val_filelists)))

Total Nums: 100, train: 80, val: 20


## 3.2 数据加载

In [7]:
### 从训练集文件夹中加载每个图像和对应的黄金标准

class FundusDataset(Dataset):
    def __init__(self, image_file, gt_file=None, filelists=None,  mode='train'):
        super(FundusDataset, self).__init__()
        self.mode = mode
        self.image_path = image_file
        image_idxs = os.listdir(self.image_path)
        self.gt_file = gt_file

        if self.mode == 'train':
            label = {str(int(row['data'])).zfill(4)+'.jpg': row[1:].values 
                        for _, row in pd.read_excel(gt_file).iterrows()}
            self.file_list = [[image_idxs[i], label[image_idxs[i]]] for i in range(len(image_idxs))]
        
        elif self.mode == 'test':
            self.file_list = [[image_idxs[i], None] for i in range(len(image_idxs))]
        
        if filelists is not None:
            self.file_list = [item for item in self.file_list if item[0] in filelists] 
   
    def __getitem__(self, idx):
        real_index, label = self.file_list[idx]
        fundus_img_path = os.path.join(self.image_path, real_index)
        fundus_img = cv2.imread(fundus_img_path)[:, :, ::-1] # BGR -> RGB        
        h,w,c = fundus_img.shape
        if self.mode == 'train':
            label_nor = (float(label[0])/w, float(label[1])/h)
            label_nor = np.array(label_nor).astype('float32').reshape(2)
        fundus_re = cv2.resize(fundus_img,(image_size, image_size))
        img = fundus_re.transpose(2, 0, 1) # H, W, C -> C, H, W
        # print(img.shape)
        # img = fundus_re.astype(np.float32)
        
        if self.mode == 'test':
            return img, real_index, h, w
        if self.mode == 'train':
            return img, label_nor

    def __len__(self):
        return len(self.file_list)


# 四、模型构建
## 4.1 模型组网

* 本案例尝试使用resnet152进行模型组网，resnet152相比于resnet50对于本赛题更为适用
* 详情可参考[官方文档](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/02_paddle2.0_develop/04_model_cn.html)组建神经网络模型

In [8]:
class Network(paddle.nn.Layer):
    def __init__(self):
        super(Network, self).__init__()
        self.resnet = paddle.vision.models.resnet152(pretrained=True, num_classes=0)
        self.flatten = paddle.nn.Flatten()
        self.linear_1 = paddle.nn.Linear(2048, 512) #线性变换层(输入单元的数目,输出单元数目)
        self.linear_2 = paddle.nn.Linear(512, 256)
        self.linear_3 = paddle.nn.Linear(256, 2)
        self.relu = paddle.nn.ReLU()   #ReLU激活层
        self.dropout = paddle.nn.Dropout(0.2) #Dropout是一种正则化手段，该算子根据给定的丢弃概率 p ，在训练过程中随机将一些神经元输出设置为0，通过阻止神经元节点间的相关性来减少过拟合。
    
    def forward(self, inputs):
        # print('input', inputs)
        y = self.resnet(inputs)
        y = self.flatten(y)
        y = self.linear_1(y)
        y = self.linear_2(y)
        y = self.relu(y)
        y = self.dropout(y)
        y = self.linear_3(y)
        y = paddle.nn.functional.sigmoid(y)

        return y

## 4.2 定义功能函数

In [9]:
# 计算欧式距离
def cal_ed(logit, label):
    ed_loss = []
    for i in range(logit.shape[0]):
        logit_tmp = logit[i,:].numpy()
        label_tmp = label[i,:].numpy()
        # print('cal_coordinate_loss_ed', logit_tmp, label_tmp)        
        ed_tmp = euclidean_distances([logit_tmp], [label_tmp])
        # print('ed_tmp:', ed_tmp[0][0])
        ed_loss.append(ed_tmp)
    
    ed_l = sum(ed_loss)/len(ed_loss)
    return ed_l

In [10]:
# 验证过程中计算欧式距离
def cal_ed_val(logit, label):
    ed_loss = []
    for i in range(logit.shape[0]):
        logit_tmp = logit[i,:]
        label_tmp = label[i,:]
        ed_tmp = euclidean_distances([logit_tmp], [label_tmp])
        ed_loss.append(ed_tmp)
    
    ed_l = sum(ed_loss)/len(ed_loss)
    
    return ed_l

## 4.3 定义损失函数

In [11]:
# 损失函数
def cal_coordinate_Loss(logit, label, alpha = 0.5):
    """
    logit: shape [batch, ndim]
    label: shape [batch, ndim]
    ndim = 2 represents coordinate_x and coordinaate_y
    alpha: weight for MSELoss and 1-alpha for ED loss
    return: combine MSELoss and ED Loss for x and y, shape [batch, 1]
    """
    alpha = alpha
    mse_loss = nn.MSELoss(reduction='mean')

    mse_x = mse_loss(logit[:,0],label[:,0])
    mse_y = mse_loss(logit[:,1],label[:,1])
    mse_l = 0.5*(mse_x + mse_y)
    # print('mse_l', mse_l)

    ed_loss = []
    # print(logit.shape[0])
    for i in range(logit.shape[0]):
        logit_tmp = logit[i,:].numpy()
        label_tmp = label[i,:].numpy()
        # print('cal_coordinate_loss_ed', logit_tmp, label_tmp)        
        ed_tmp = euclidean_distances([logit_tmp], [label_tmp])
        # print('ed_tmp:', ed_tmp[0][0])
        ed_loss.append(ed_tmp)
    
    ed_l = sum(ed_loss)/len(ed_loss)
    # print('ed_l', ed_l)
    # print('alpha', alpha)
    loss = alpha * mse_l + (1-alpha) * ed_l
    # print('loss in function', loss)
    return loss

## 4.4 定义训练函数及验证函数

In [12]:
### 训练函数

def train(model, iters, train_dataloader, val_dataloader, optimizer, log_interval, evl_interval):
    iter = 0
    model.train()
    avg_loss_list = []
    avg_ED_list = []
    best_ED = sys.float_info.max
    while iter < iters:
        for img, lab in train_dataloader:
            iter += 1
            if iter > iters:
                break
            fundus_imgs = (img / 255.).astype('float32')
            label = lab.astype("float32")

            logits = model(fundus_imgs)
            loss = cal_coordinate_Loss(logits, label)
            # print('loss in train',loss)

            for p,l in zip(logits.numpy(), label.numpy()):
                avg_ED_list.append([p,l])
            
            # print('avg_ED_list', avg_ED_list)
            loss.backward()
            optimizer.step()
            model.clear_gradients()
            avg_loss_list.append(loss.numpy()[0])
            
            if iter % log_interval == 0:
                avg_loss = np.array(avg_loss_list).mean()
                # print(avg_loss)
                avg_ED_list = np.array(avg_ED_list)
                avg_ED = cal_ed_val(avg_ED_list[:, 0], avg_ED_list[:, 1]) # cal_ED
                # print('ed in training', avg_ED)
                avg_loss_list = []
                avg_ED_list = []
                
                print("[TRAIN] iter={}/{} avg_loss={:.4f} avg_ED={:.4f}".format(iter, iters, avg_loss, avg_ED[0][0]))

            if iter % evl_interval == 0:
                avg_loss, avg_ED = val(model, val_dataloader)
                print("[EVAL] iter={}/{} avg_loss={:.4f} ED={:.4f}".format(iter, iters, avg_loss, avg_ED[0][0]))
                if avg_ED <= best_ED:
                    best_ED = avg_ED[0][0]
                    paddle.save(model.state_dict(),
                            os.path.join("best_model_{:.4f}".format(best_ED), 'model.pdparams'))
                model.train()

### 验证函数

def val(model, val_dataloader):
    model.eval()
    avg_loss_list = []
    cache = []
    with paddle.no_grad():
        for data in val_dataloader:
            fundus_imgs = (data[0] / 255.).astype("float32")
            labels = data[1].astype('float32')
            
            logits = model(fundus_imgs)
            for p, l in zip(logits.numpy(), labels.numpy()):
                cache.append([p, l])

            loss = cal_coordinate_Loss(logits, labels)
            avg_loss_list.append(loss.numpy()[0])

    cache = np.array(cache)
    ED = cal_ed_val(cache[:, 0], cache[:, 1])
    avg_loss = np.array(avg_loss_list).mean()

    return avg_loss, ED

# 五、训练、评估与预测
## 5.1 生成训练集和验证集 

In [13]:
### 生成训练集和验证集 

train_dataset = FundusDataset(image_file = images_file, 
                       gt_file=gt_file,
                       filelists=train_filelists)

val_dataset = FundusDataset(image_file = images_file, 
                       gt_file=gt_file,
                       filelists=val_filelists)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xlrd/xlsx.py:39: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  try: import defusedxml.cElementTree as ET


In [14]:
### 加载数据

train_loader = paddle.io.DataLoader(
    train_dataset,
    batch_sampler=paddle.io.DistributedBatchSampler(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False),
    # num_workers=num_workers,
    return_list=True,
    use_shared_memory=False
)

val_loader = paddle.io.DataLoader(
    val_dataset,
    batch_sampler=paddle.io.DistributedBatchSampler(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False),
    # num_workers=num_workers,
    return_list=True,
    use_shared_memory=False
)

In [15]:

model = Network()

if optimizer_type == "adam":
    optimizer = paddle.optimizer.Adam(init_lr, parameters=model.parameters())

# criterion = cal_coordinate_Loss()

W1230 15:52:54.637707   102 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1230 15:52:54.640895   102 device_context.cc:372] device: 0, cuDNN Version: 7.6.
2021-12-30 15:52:58,958 - INFO - unique_endpoints {''}
2021-12-30 15:52:58,960 - INFO - Downloading resnet152.pdparams from https://paddle-hapi.bj.bcebos.com/models/resnet152.pdparams
100%|██████████| 355826/355826 [00:04<00:00, 73990.22it/s]
2021-12-30 15:53:03,885 - INFO - File /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams md5 checking...


## 5.2 训练模型 
* 使用以下命令正式开始训练
* 训练轮数为3000轮

In [16]:
### 训练过程
iters = 3000
evl_interval = iters//5

train(model, iters, train_loader, val_loader, optimizer, log_interval=10, evl_interval=100)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, u

[TRAIN] iter=10/3000 avg_loss=0.0132 avg_ED=0.2098


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, u

[TRAIN] iter=20/3000 avg_loss=0.0044 avg_ED=0.1145


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, u

...

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, u

[TRAIN] iter=2990/3000 avg_loss=0.0000 avg_ED=0.0081


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, u

[TRAIN] iter=3000/3000 avg_loss=0.0000 avg_ED=0.0078
[EVAL] iter=3000/3000 avg_loss=0.0001 ED=0.0132


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):


## 5.3 模型评估及预测 
* 最终比赛提交的结果中，checkpoints使用的是/home/aistudio/best_model_0.0124/路径下的**model.pdparams**

In [17]:
### 测试过程，加载模型参数

best_model_path = "./best_model_0.0124/model.pdparams"
model = Network()
para_state_dict = paddle.load(best_model_path)
model.set_state_dict(para_state_dict)
model.eval()

2021-12-30 18:08:57,684 - INFO - unique_endpoints {''}
2021-12-30 18:08:57,685 - INFO - File /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams md5 checking...
2021-12-30 18:08:58,471 - INFO - Found /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams


In [18]:
### 生成测试集

test_dataset = FundusDataset(image_file = test_file, mode='test')

In [19]:
### 一张一张测试样本

cache = []
for fundus_img, idx, h, w in test_dataset:
    fundus_img = fundus_img[np.newaxis, ...]    
    fundus_img = paddle.to_tensor((fundus_img / 255.).astype("float32"))    
    logits = model(fundus_img)
    pred_coor = logits.numpy()
    # print(pred_coor)
    x = pred_coor[0][0] * w
    y = pred_coor[0][1] * h
    cache.append([idx.split('.')[0], x, y])

* 将预测结果写入Localization_Results.csv文件中

In [20]:
### 将所有测试集中预测结果存到.csv中

submission_result = pd.DataFrame(cache, columns=['data', 'Fovea_X', 'Fovea_Y'])
submission_result[['data', 'Fovea_X', 'Fovea_Y']].to_csv("./Localization_Results.csv", index=False)

# 总结及展望

* 尝试其他优化器
* 进一步调整学习率及超参数，考虑一些学习率调整策略
* 尝试更深层的网络模型



# 参考资料
[飞桨常规赛：黄斑中央凹定位（GAMMA挑战赛任务二） - 10月第1名方案](https://aistudio.baidu.com/aistudio/projectdetail/2548493?channelType=0&channel=0)

[GAMMA挑战赛任务二基线](https://aistudio.baidu.com/aistudio/projectdetail/2255726?channelType=0&channel=0)